In [1]:
import os

In [2]:
%pwd

'/home/shrey/Desktop/Kidney-Disease-Classifcation/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/shrey/Desktop/Kidney-Disease-Classifcation'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list

In [6]:
os.chdir("./src")

In [7]:
from kidney_classification.constants import *
from kidney_classification.utils.common import read_yaml, create_directories
import tensorflow as tf

2024-01-03 15:58:59.001906: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-03 15:58:59.003769: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-03 15:58:59.032703: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-03 15:58:59.032753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-03 15:58:59.033922: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        img_height, img_width = self.config.params_image_size[:-1]

        train = tf.keras.utils.image_dataset_from_directory(
            self.config.training_data,
            image_size=(img_height, img_width),
            validation_split=0.1,
            subset='training',
            seed=123
        )
        
        val = tf.keras.utils.image_dataset_from_directory(
            self.config.training_data,
            image_size=(img_height, img_width),
            validation_split=0.2,
            subset='validation',
            seed=123
        )
        train = train.map(lambda x, y: (x / 255, y))
        val = val.map(lambda x, y: (x / 255, y))
        AUTOTUNE = tf.data.AUTOTUNE

        self.train_dataset = train.cache().prefetch(buffer_size=AUTOTUNE)
        self.val_dataset = val.cache().prefetch(buffer_size=AUTOTUNE)

        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def define_and_train_model(self):

        self.model.fit(
            self.train_dataset,
            validation_data=self.val_dataset,
            epochs=self.config.params_epochs,
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.15
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
os.chdir("../")

In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.define_and_train_model()
    
except Exception as e:
    raise e

[2024-01-03 15:59:11,299: INFO: common yaml file: config/config.yaml loaded successfully]
[2024-01-03 15:59:11,301: INFO: common yaml file: params.yaml loaded successfully]


2024-01-03 15:59:11.602362: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


[2024-01-03 15:59:12,070: WARNING: optimizer Skipping variable loading for optimizer 'Adam', because it has 13 variables whereas the saved optimizer has 1 variables. ]
Found 12446 files belonging to 4 classes.
Using 11202 files for training.
Found 12446 files belonging to 4 classes.
Using 2489 files for validation.
Epoch 1/15
351/351 [==============================] - 561s 2s/step - loss: 0.5099 - accuracy: 0.8103 - val_loss: 0.2366 - val_accuracy: 0.9152
Epoch 2/15
351/351 [==============================] - 566s 2s/step - loss: 0.2120 - accuracy: 0.9229 - val_loss: 0.2941 - val_accuracy: 0.8803
Epoch 3/15
351/351 [==============================] - 571s 2s/step - loss: 0.1573 - accuracy: 0.9466 - val_loss: 0.0994 - val_accuracy: 0.9667
Epoch 4/15
351/351 [==============================] - 547s 2s/step - loss: 0.1370 - accuracy: 0.9528 - val_loss: 0.0742 - val_accuracy: 0.9787
Epoch 5/15
351/351 [==============================] - 552s 2s/step - loss: 0.1217 - accuracy: 0.9565 - val_loss

/home/shrey/Desktop/Kidney-Disease-Classifcation/env/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
